In [ ]:
import syft_runtimes.high_low as syhl
from syft_core import Client
import syft_datasets as syd
from pathlib import Path

## Initialize the high datasite (secure, self-contained)

In [ ]:
highlow_identifier = "highlow-1234"

In [ ]:
client = Client.load()

highside_client: syhl.HighSideClient = syhl.initialize_high_datasite(
    email=client.email,
    highlow_identifier=highlow_identifier,
    force_overwrite=True,
)

## Create dataset on high side

In [ ]:
DATASET_NAME = "my_cool_dataset"
data_dir = Path("./data")

highside_dataset = syd.create(
    name=DATASET_NAME,
    mock_path=data_dir / "mock_data.csv",
    private_path=data_dir / "private_data.csv",
    readme_path=data_dir / "readme.md",
    summary="This is a mock dataset for demonstration purposes.",
    tags=["example", "testing", "highside"],
    location=highlow_identifier,
)

## Configure secure connection to low side

Only SSH parameters - no low side file system knowledge required


In [ ]:
ssh_config: dict = {
    "low_ssh_host": "74.235.86.18",
    "low_ssh_user": "azureuser",
    "low_ssh_port": 22,
    "low_ssh_key_path": "~/.ssh/flower-vm-1",
}

lowside_client = highside_client.lowside_connect(
    highlow_identifier=highlow_identifier, connection_config=ssh_config
)

In [ ]:
# Sync the dataset (mock part) to the low side
highside_client.sync_dataset(
    dataset_name=DATASET_NAME,
    verbose=True,
)

In [ ]:
highside_client.sync_pending_jobs()

In [ ]:
highside_client.sync_done_jobs()